In [ ]:
import numpy as np 
import pandas as pd 
import os
from glob import glob
from random import shuffle
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution1D, concatenate, SpatialDropout1D, GlobalMaxPool1D, GlobalAvgPool1D, Embedding, \
    Conv2D, SeparableConv1D, Add, BatchNormalization, Activation, GlobalAveragePooling2D, LeakyReLU, Flatten
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, \
    Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D
from keras.layers.pooling import _GlobalPooling1D
from keras.losses import mae, sparse_categorical_crossentropy, binary_crossentropy
from keras.models import Model
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import auc

In [5]:
import methods as md
#train_labels, id_label_map, get_id_from_file_path(file_path)

In [8]:
labeled_files = glob('./dataset/training/*.tif')
train, val = train_test_split(labeled_files, test_size=0.2, random_state=101010)

In [16]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [20]:
def data_gen(list_files, id_label_map, batch_size, augment=False):
    keras_gen = ImageDataGenerator(
                    rotation_range=10,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    horizontal_flip=True,
                    vertical_flip=True,
                    zoom_range=0.2,
                    shear_range=5)
    while True:
        shuffle(list_files)
        for batch in chunker(list_files, batch_size):
            X = [cv2.imread(x) for x in batch]
            Y = [id_label_map[md.get_id_from_file_path(x)] for x in batch]
            if augment:
                X = [keras_gen.random_transform(x) for x in X]
            X = [preprocess_input(x.astype(np.float32)) for x in X]
                
            yield np.array(X), np.array(Y)

In [10]:
def get_model_classif_mobilenet2():
    inputs = Input((96, 96, 3))
    base_model = MobileNetV2(include_top=False, input_shape=(96, 96, 3))#, weights=None
    x = base_model(inputs)
    out1 = GlobalMaxPooling2D()(x)
    out2 = GlobalAveragePooling2D()(x)
    out3 = Flatten()(x)
    out = Concatenate(axis=-1)([out1, out2, out3])
    out = Dropout(0.5)(out)
    out = Dense(1, activation="sigmoid", name="3_")(out)
    model = Model(inputs, out)
    model.compile(optimizer=Adam(0.0001), loss=binary_crossentropy, metrics=['acc'])
    model.summary()

    return model

In [11]:
model = get_model_classif_mobilenet2()

Instructions for updating:
Colocations handled automatically by placer.
9412608/9406464 [==============================] - 3s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
mobilenetv2_1.00_96 (Model)     (None, 3, 3, 1280)   2257984     input_1[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_1 (GlobalM (None, 1280)         0           mobilenetv2_1.00_96[1][0]        
_________________________________________________________

In [12]:
batch_size=32
h5_path = "model.h5"
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [ ]:
history = model.fit_generator(
    data_gen(train, md.id_label_map, batch_size, augment=True),
    validation_data=data_gen(val, md.id_label_map, batch_size),
    epochs=2, verbose=1,
    callbacks=[checkpoint],
    steps_per_epoch=len(train) // batch_size,
    validation_steps=len(val) // batch_size)
batch_size=64
history = model.fit_generator(
    data_gen(train, md.id_label_map, batch_size, augment=True),
    validation_data=data_gen(val, md.id_label_map, batch_size),
    epochs=6, verbose=1,
    callbacks=[checkpoint],
    steps_per_epoch=len(train) // batch_size,
    validation_steps=len(val) // batch_size)
    
model.compile(optimizer=Adam(0.00001), loss=binary_crossentropy, metrics=['acc'])
history = model.fit_generator(
    data_gen(train, md.id_label_map, batch_size, augment=True),
    validation_data=data_gen(val, md.id_label_map, batch_size),
    epochs=6, verbose=1,
    callbacks=[checkpoint],
    steps_per_epoch=len(train) // batch_size,
    validation_steps=len(val) // batch_size)

Epoch 1/2
5500/5500 [==============================] - 6807s 1s/step - loss: 0.3295 - acc: 0.8711 - val_loss: 0.3049 - val_acc: 0.8874

Epoch 00001: val_acc improved from -inf to 0.88736, saving model to model.h5
Epoch 2/2
5500/5500 [==============================] - 6590s 1s/step - loss: 0.2106 - acc: 0.9195 - val_loss: 0.2475 - val_acc: 0.9157

Epoch 00002: val_acc improved from 0.88736 to 0.91565, saving model to model.h5
Epoch 1/6
2750/2750 [==============================] - 6635s 2s/step - loss: 0.1574 - acc: 0.9411 - val_loss: 0.1929 - val_acc: 0.9341

Epoch 00001: val_acc improved from 0.91565 to 0.93407, saving model to model.h5
Epoch 2/6
  17/2750 [..............................] - ETA: 1:49:59 - loss: 0.1652 - acc: 0.9390

In [15]:
model.load_weights(h5_path)

preds = []
ids = []

for batch in chunker(test_files, batch_size):
    X = [preprocess_input(cv2.imread(x).astype(np.float32)) for x in batch]
    ids_batch = [get_id_from_file_path(x) for x in batch]
    X = np.array(X)
    preds_batch = ((model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
    preds += preds_batch
    ids += ids_batch

In [ ]:
df = pd.DataFrame({'id':ids, 'label':preds})
df.to_csv("baseline_mobilenet_v2.csv", index=False)
df.head()

In [ ]:
fpr_keras, tpr_keras, thresholds_keras = roc_curve(val, preds)

In [ ]:
auc_keras = auc(fpr_keras, tpr_keras)